# Модель кредитного скоринга: предобработка данных.

**Описание задачи**: 

Банку требуется создать модель кредитного скоринга, для этого нужно провести преобработку данных о клиентах. 

Помимо обработки данных выделю новые признаки для построения модели, а также проведу первичный анализ для составления портрета заемщика, склонного к задолженности.

## Обзор данных

Проведу обзор данных, посмотрим, какие типы данных в датафрейме:

In [ ]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
display(data)
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Обзор данных показывает наличие пропусков в столбцах days_employed и total_income, возможно это связано с тем, что некоторые клиенты просто не указывали эти данные. Но при этом количество пропусков одинаковое, возможно у части клиентов не запрашивали данные по этим категориям. Далее я заполню данные медианным значение, что допустимо для количественных данных. Для большей точности предпочтительно ориентироваться на данные не только конкрентного столбца, но и смежных столбцов, например, таким образом я заполнял пропуски в столбце total_income в зависимости от разных типо дохода, содержащихся в стоблце income_type. 

## Заполнение пропусков и проверка данных на аномалии

Посчитаю пропуски в данных и их долю.

In [ ]:
print(data.isna().sum())
print(f'Доля пропущенных значений: {2174/21525:.2%}') # это доля пропущенных значений, которые были

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
Доля пропущенных значений: 10.10%


Я проверил, что в датафрейме есть отрицательные значения, хотя их быть не должно по логике данных.

In [ ]:
display(data[data['days_employed'] < 0])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
...,...,...,...,...,...,...,...,...,...,...,...,...
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Заменим медианным значением, но сначала удалим отрицательные значения.

In [ ]:
data['days_employed'] = abs(data['days_employed'])
display(data) #убираем отрицательные значения, те. приводим к модулю

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Заменим медианным значением для каждого типа дохода:

In [ ]:
income_types = data['income_type'].unique()
for type in income_types:
    data['days_employed'] = data['days_employed'].fillna(data[data['income_type'] == type]['days_employed'].median())
    data['total_income'] = data['total_income'].fillna(data[data['income_type'] == type]['total_income'].median())

In [ ]:
data.info() #проверяем

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Еще в столбце 'children' есть отрицательное значение, изменим его на положительное.
А также примем, что 20 детей у заемщика это опечатка, и заменим эти значения на '2'.

In [ ]:
data['children'].unique()
data['children'] = abs(data['children'])
try:
    data.loc[data['children'] == 20, 'children'] = 2
except:
    print('20 не поменялось на 2')

В столбце 'days_employed' очень большие значения, из условия я знаю, что это количество дней.

In [ ]:
data['days_employed'].sort_values()

17437        24.141633
8336         24.240695
6157         30.195337
9683         33.520665
2127         34.701045
             ...      
7794     401663.850046
2156     401674.466633
7664     401675.093434
10006    401715.811749
6954     401755.400475
Name: days_employed, Length: 21525, dtype: float64

Применим несколько методов к столбцу days_employed для выявления возможных аномалий.

In [ ]:
data_check = data.groupby('income_type').agg({'days_employed': ['median', 'mean', 'max', 'min', 'count']})
display(data_check) # Получается, что самые аномальные цифры у пенсионеров и безработных

days_employed                                               \
                        median           mean            max            min   
income_type                                                                   
безработный      366413.652744  366413.652744  395302.838654  337524.466835   
в декрете          3296.759962    3296.759962    3296.759962    3296.759962   
госслужащий        2385.358043    3215.951028   15193.032201      39.954170   
компаньон          1574.202821    2057.845075   17615.563266      30.195337   
пенсионер        360505.668544  326078.103247  401755.400475    1574.202821   
предприниматель    1047.525452    1047.525452    1574.202821     520.848083   
сотрудник          1574.202821    2251.736421   18388.949901      24.141633   
студент             578.751554     578.751554     578.751554     578.751554   

                        
                 count  
income_type             
безработный          2  
в декрете            1  
госслужащий       1459  
компаньон         5085  
пенсионер         3856  
предприниматель      2  
сотрудник        11119  
студент              1

Дополнительной информации у нас нет, но 1000 лет стажа нет даже в самых крутых резюме.
Я могу предположить, что здесь указаны часы. 
Переведу у пенсионеров и безработных часы в дни, а потом создам колонку со стажем в годах для удобства.

In [ ]:
data.loc[data['income_type'] == 'пенсионер', 'days_employed'] /= 24
data.loc[data['income_type'] == 'безработный', 'days_employed'] /= 24
data['years_employed'] = data['days_employed'] / 365
display(data)
#в столбце с годами стажа нет явных аномальных значений
display(data['years_employed'].sort_values())


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,38.843159
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,12.409087
21521,0,14330.725172,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем,39.262261
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,5.789991
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,8.527347


17437     0.066141
8336      0.066413
6157      0.082727
9683      0.091837
2127      0.095071
           ...    
7664     45.853321
10006    45.857969
6954     45.862489
4299     48.261817
16335    50.380685
Name: years_employed, Length: 21525, dtype: float64

В столбце dob_years содержится возраст 0, это ошибка.
Также посчитаю, в скольких строках указан возраст 0.

In [ ]:
print(data['dob_years'].unique())
print(data[data['dob_years'] == 0].count())

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
children            101
days_employed       101
dob_years           101
education           101
education_id        101
family_status       101
family_status_id    101
gender              101
income_type         101
debt                101
total_income        101
purpose             101
years_employed      101
dtype: int64


Заменю на медианное значение по каждой группе:

In [ ]:
for type in income_types:
    data.loc[(data['dob_years'] == 0) & (data['income_type'] == type), 'dob_years'] = data[data['income_type'] == type]['dob_years'].median()
print(data['dob_years'].unique())

[42. 36. 33. 32. 53. 27. 43. 50. 35. 41. 40. 65. 54. 56. 26. 48. 24. 21.
 57. 67. 28. 63. 62. 47. 34. 68. 25. 31. 30. 20. 49. 37. 45. 61. 64. 44.
 52. 46. 23. 38. 39. 51. 60. 59. 29. 55. 58. 71. 22. 73. 66. 69. 19. 72.
 70. 74. 75.]


## Изменение типов данных

Для удобства подсчета и визуализации изменю тип данных с вещественного на целочисленный в столбцах total_income, dob_years и years_employed.

In [ ]:
data['total_income'] = data['total_income'].astype('int')
data['dob_years'] = data['dob_years'].astype('int')
data['years_employed'] = data['years_employed'].astype('int')
display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,12
21521,0,14330.725172,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,39
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,5
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,8


## Удаление дубликатов

Проверю датафрейм на наличие дубликатов и выведу строки с дубликатами.

In [ ]:
print(data.duplicated().sum())
display(data[data.duplicated()])

54


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
2849,0,1574.202821,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи,4
4182,1,1574.202821,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,142594,свадьба,4
4851,0,65.591784,60,среднее,1,гражданский брак,1,F,пенсионер,0,142594,свадьба,0
5557,0,65.591784,58,среднее,1,гражданский брак,1,F,пенсионер,0,142594,сыграть свадьбу,0
7808,0,65.591784,57,среднее,1,гражданский брак,1,F,пенсионер,0,142594,на проведение свадьбы,0
8583,0,65.591784,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,142594,дополнительное образование,0
9238,2,1574.202821,34,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для сдачи,4
9528,0,65.591784,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,142594,операции со своей недвижимостью,0
9627,0,65.591784,56,среднее,1,женат / замужем,0,F,пенсионер,0,142594,операции со своей недвижимостью,0
10462,0,65.591784,62,среднее,1,женат / замужем,0,F,пенсионер,0,142594,покупка коммерческой недвижимости,0


Здесь требуется избавится от неявных дубликатов, сначала посмотрю спиксок уникальных значений.
Затем перезапишу столбец, переведя значения в нижний регистр.

In [ ]:
print(data['education'].value_counts())
data['education'] = data['education'].str.lower()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


Удалю явные дубликаты с присвоением новой индексации.

In [ ]:
data = data.drop_duplicates().reset_index(drop=True)
display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,12
21450,0,14330.725172,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,39
21451,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,5
21452,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,8


Проверю неявные дубликаты в колонке family_status.

In [ ]:
print(data['family_status'].value_counts())

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


Дубликатов нет, но исправим прописные буквы на строчные

In [ ]:
data['family_status'] = data['family_status'].str.lower()

Проверю неявные дубликаты в столбце 'purpose'.

In [ ]:
print(data['purpose'].value_counts())

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Здесь содержатся неявные дубликаты, удалю их с использованием метода .replace() и создания функции.
В дальнейшем можно выделить только 4 категории: свадьба, недвижимость, автомобиль, образование. 
Но я думаю, что более детальная информация теоретически может понадобиться при дальнейшем анализе (например, коммерческая или жилая недвижимость и тд.), поэтому я устранил самые явные дубликаты.
Заодно исправлю опечатку в названии категории 'ремонт жилью'.

In [ ]:
def replace_duplicates(duplicates, correct):
    for duplicate in duplicates:
        data['purpose'] = data['purpose'].replace(duplicate,correct)
        
wedding_duplicates = ['на проведение свадьбы','сыграть свадьбу']
wedding_correct = 'свадьба'
replace_duplicates(wedding_duplicates, wedding_correct)

commercial_property_duplicates = ['операции с коммерческой недвижимостью']
commercial_property_correct = 'покупка коммерческой недвижимости'
replace_duplicates(commercial_property_duplicates, commercial_property_correct)

real_estate_duplicates = ['покупка жилья', 'покупка жилья для семьи', 'покупка своего жилья', 'покупка жилой недвижимости', 'операции с жильем']
real_estate_correct = 'жилье'
replace_duplicates(real_estate_duplicates, real_estate_correct)

property_operation_duplicates = ['операции со своей недвижимостью', 'операции с недвижимостью', 'покупка недвижимости']
property_operation_correct = 'недвижимость'                               
replace_duplicates(property_operation_duplicates, property_operation_correct)

construction_duplicates = 'строительство собственной недвижимости'
construction_correct = 'строительство недвижимости'
replace_duplicates(construction_duplicates, construction_correct)

auto_duplicates = ['на покупку своего автомобиля', 'свой автомобиль', 'автомобили', 'на покупку автомобиля', 'приобретение автомобиля', 'сделка с автомобилем']
auto_correct = 'автомобиль'
replace_duplicates(auto_duplicates, auto_correct)

used_auto_duplicates = ['на покупку подержанного автомобиля']
used_auto_correct = 'сделка с подержанным автомобилем'
replace_duplicates(used_auto_duplicates, used_auto_correct)

education_duplicates = ['получение образования', 'заняться образованием']
education_correct = 'образование'
replace_duplicates(education_duplicates, education_correct)

higher_education_duplicates = ['получение высшего образования', 'заняться высшим образованием']
higher_education_correct = 'высшее образование'
replace_duplicates(higher_education_duplicates, higher_education_correct)

add_education_duplicates = ['получение дополнительного образования']
add_education_correct = 'дополнительное образование'
replace_duplicates(add_education_duplicates, add_education_correct)

data.loc[data['purpose'] == 'ремонт жилью', 'purpose'] = 'ремонт жилья'

In [ ]:
print(data['purpose'].value_counts()) # проверим результат

жилье                                     3808
автомобиль                                3342
недвижимость                              2556
свадьба                                   2324
высшее образование                        1374
покупка коммерческой недвижимости         1311
образование                               1297
сделка с подержанным автомобилем           964
дополнительное образование                 906
покупка жилья для сдачи                    651
строительство собственной недвижимости     635
строительство жилой недвижимости           624
строительство недвижимости                 619
ремонт жилья                               607
профильное образование                     436
Name: purpose, dtype: int64


## Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

Выделю 2 новых датафрейма из исходного, удалю дубликаты и сброшу индексы.
Также удалю из исходного датафрейма столбцы 'education' и 'family_status', чтобы обращаться к значениям по идентификаторам через созданные словари.

In [ ]:
data_education = data[['education', 'education_id']]
data_family = data[['family_status', 'family_status_id']]
data_education = data_education.drop_duplicates().reset_index(drop=True)
data_family = data_family.drop_duplicates().reset_index(drop=True)

data = data[['children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose', 'years_employed']]
display(data)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,жилье,23
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,автомобиль,11
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,жилье,15
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,11
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,свадьба,38
...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,1,1,F,компаньон,0,224791,жилье,12
21450,0,14330.725172,67,1,0,F,пенсионер,0,155999,автомобиль,39
21451,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость,5
21452,3,3112.481705,38,1,0,M,сотрудник,1,244093,автомобиль,8


##  Категоризация дохода

Проведу классификацию заемщиков по уровню дохода, для этого напишу функцию, и результат будет указан в новом столбце.

In [ ]:
def total_income_category(income):
    if income <= 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    if income >= 1000001:
        return 'A'
data['total_income_category'] = data['total_income'].apply(total_income_category)
display(data)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,жилье,23,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,автомобиль,11,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,жилье,15,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,11,B
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,свадьба,38,C
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,1,1,F,компаньон,0,224791,жилье,12,B
21450,0,14330.725172,67,1,0,F,пенсионер,0,155999,автомобиль,39,C
21451,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость,5,C
21452,3,3112.481705,38,1,0,M,сотрудник,1,244093,автомобиль,8,B


## Категоризация целей кредита

Создам столбец 'purpose_category', в котором будет 4 вида категорий: 'операции с автомобилем','операции с недвижимостью','проведение свадьбы','получение образования'. Это нужно для дальнейшего выявления зависимости наличия задолженности от общей цели взятия кредита.

In [ ]:
def purpose_type(purpose):
    if purpose in ['жилье', 'недвижимость', 'покупка коммерческой недвижимости', 'покупка жилья для сдачи', 'строительство собственной недвижимости', 'строительство жилой недвижимости', 'строительство недвижимости', 'ремонт жилья']:
        return 'операции с недвижимостью'
    if purpose in ['автомобиль', 'сделка с подержанным автомобилем']:
        return 'операции с автомобилем'
    if purpose == 'свадьба':
        return 'проведение свадьбы'
    if purpose in ['высшее образование', 'дополнительное образование', 'профильное образование', 'образование']:
        return 'получение образования'
data['purpose_category'] = data['purpose'].apply(purpose_type)
display(data)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,жилье,23,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,автомобиль,11,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,жилье,15,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,11,B,получение образования
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,свадьба,38,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,1,1,F,компаньон,0,224791,жилье,12,B,операции с недвижимостью
21450,0,14330.725172,67,1,0,F,пенсионер,0,155999,автомобиль,39,C,операции с автомобилем
21451,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость,5,C,операции с недвижимостью
21452,3,3112.481705,38,1,0,M,сотрудник,1,244093,автомобиль,8,B,операции с автомобилем


## Портрет клиентов, имеющих задолженность

У нас есть информация в столбце 'debt', имеется ли задолженность по возврату кредитов (0 или 1)
Я решил добавить в основной датафрейм таблицу с семейным статусом.

In [ ]:
data_new = data.merge(data_family, on='family_status_id', how='left')

Составлю несколько сводных таблиц, для понимания того, как характеристики заемщиков влияют на наличие задолженности по кредиту. Посмотрим, сколько заемщиков в каждой категории: по количеству детей, по семейному статусу, по доходу и по цели кредита.
Также выведем средний долг по каждой категории.

In [ ]:
display(data_new.pivot_table(index='children', values='debt', aggfunc=['mean','count']))
display(data_new.pivot_table(index='family_status', values='debt', aggfunc=['mean','count']))
display(data_new.pivot_table(index='total_income_category', values='debt', aggfunc=['mean','count']))
display(data_new.pivot_table(index='purpose_category', values='debt', aggfunc=['mean','count']))

,mean,count
,debt,debt
children,,
0,0.075438,14091
1,0.091658,4855
2,0.094925,2128
3,0.081818,330
4,0.097561,41
5,0.000000,9


,mean,count
,debt,debt
family_status,,
в разводе,0.071130,1195
вдовец / вдова,0.065693,959
гражданский брак,0.093471,4151
женат / замужем,0.075452,12339
не женат / не замужем,0.097509,2810


,mean,count
,debt,debt
total_income_category,,
A,0.080000,25
B,0.070621,5041
C,0.084915,16016
D,0.060000,350
E,0.090909,22


,mean,count
,debt,debt
purpose_category,,
операции с автомобилем,0.093590,4306
операции с недвижимостью,0.072334,10811
получение образования,0.092200,4013
проведение свадьбы,0.080034,2324


Создам подробную сводную таблицу для получения информации о взаимозависимости возврата кредита и следующих показателей: количества детей, семейное положение, цель кредита и уровень дохода. Таблица подробная и в некоторых подкатегориях не содержится данных.

In [ ]:
pd.options.display.max_rows = 100
display(data_new.pivot_table(index=['family_status', 'total_income_category', 'purpose_category'], columns='children', values='debt', aggfunc='mean'))

children                                                                     0  \
family_status         total_income_category purpose_category                     
в разводе             A                     операции с недвижимостью  0.000000   
                      B                     операции с автомобилем    0.020408   
                                            операции с недвижимостью  0.063063   
                                            получение образования     0.142857   
                      C                     операции с автомобилем    0.083916   
                                            операции с недвижимостью  0.070064   
                                            получение образования     0.071429   
                      D                     операции с автомобилем    0.000000   
                                            операции с недвижимостью  0.000000   
                                            получение образования     0.000000   
вдовец / вдова        B                     операции с автомобилем    0.071429   
                                            операции с недвижимостью  0.038961   
                                            получение образования     0.080000   
                      C                     операции с автомобилем    0.101796   
                                            операции с недвижимостью  0.049738   
                                            получение образования     0.067114   
                      D                     операции с автомобилем    0.000000   
                                            операции с недвижимостью  0.000000   
                                            получение образования     0.000000   
гражданский брак      A                     операции с недвижимостью  0.000000   
                                            получение образования     1.000000   
                                            проведение свадьбы        0.000000   
                      B                     операции с автомобилем    0.058824   
                                            операции с недвижимостью  0.054422   
                                            получение образования     0.074627   
                                            проведение свадьбы        0.062857   
                      C                     операции с автомобилем    0.110048   
                                            операции с недвижимостью  0.085540   
                                            получение образования     0.148718   
                                            проведение свадьбы        0.079792   
                      D                     операции с автомобилем    0.200000   
                                            операции с недвижимостью  0.100000   
                                            получение образования     0.000000   
                                            проведение свадьбы        0.043478   
                      E                     получение образования     0.000000   
                                            проведение свадьбы        0.000000   
женат / замужем       A                     операции с автомобилем         NaN   
                                            операции с недвижимостью  0.000000   
                                            получение образования     0.000000   
                      B                     операции с автомобилем    0.077320   
                                            операции с недвижимостью  0.062124   
                                            получение образования     0.055249   
                      C                     операции с автомобилем    0.069862   
                                            операции с недвижимостью  0.067430   
                                            получение образования     0.082064   
                      D                     операции с автомобилем    0.100000   
                                            операции с недвижимо

## Общий вывод

Я провел предобработку данных, выявил аномалии, дубликаты, пропущенные значения, и вернул датафрейм в корректном виде.
Что касается вопросов исследования, на основе данных можно сделать выводы:
- Заемщики без детей в среднем чаще имеют задолженности по кредиту
- Заемщики категории 'вдовец / вдова' чаще других семейных категорий имеют задолженности по кредиту
- Заемщики, относящиеся к группе 'не женат / не замужем', реже имеют задолженности по кредиту
- Нет четкой зависимости между уровнем дохода и возвратом кредита в срок
- Заемщики, целью кредита которых являются операции с автомобилем и получение образования гораздо реже 
имеют задолженности по кредиту, чем те, чья цель проведение свадьбы или операции с недвижимостью.